# RESTful APIs - Django REST Framework 2

### Authentication & Permissions

Currently our API doesn't have any restrictions on who can edit or delete book items. We need to include in our API more advanced behavior in order to make sure that:

- Book items are always associated with a creator.
- Only authenticated users may create, update or delete book items.
- Unauthenticated requests should have just read-only access.

To begin, we're going to make a change to our `Book` model class. Let's add a field to represent the user who created a book item. Hence, add the following field to the `Book` model in `resttutorial/books/models.py`.

In [ ]:
owner = models.ForeignKey('auth.User', related_name='books')

Next, let's  delete our current database and the migrations we have a acumulated so far. If you're working with Linux, type the following in your operating system shell:

If you are working in Windows, type the following to delete the database and migration files.

Next we will re-create our database:

Create also a couple of users for testing the API. Create at least a superuser with user name: **admin** and password: **password123**. **<font color="red"> This is important since these are the credentials I will use to test your application!</font>**

Since we've got some users in our Web API to work with, we need to add a representation of those users to our API. Hence, we need to create a new serializer. In `resttutorial/books/serializers.py` add the following to the existing code:

In [ ]:
from django.contrib.auth.models import User

class UserSerializer(serializers.ModelSerializer):
    books = serializers.PrimaryKeyRelatedField(many=True, queryset=Book.objects.all())

    class Meta:
        model = User
        fields = ('id', 'username', 'books')

Because `books` is a reverse relationship on the `User` model, it will not be included by default when using the `ModelSerializer` class, so we needed to add an explicit field for it.

We'll also add a couple of views to `resttutorial/books/views.py`. We'd like to just use read-only views for the user representations. The REST framework provides predefined views that takes advantage of the fact that any model-backed API views we create are going to be very similar in terms of create/retrieve/update/delete operations.

Those bits of common behaviour are implemented in REST framework's mixin classes. With mixed-in generic views we can trim down our `resttutorial/books/views.py` module since we don't have to specify directly what to do for GET/POST/PUT/DELETE requests. That functionality is already predefined. Hence, we'll use the `ListAPIView` and `RetrieveAPIView` generic class based views. 

Add the following to your `resttutorial/books/views.py` file:

In [ ]:
from django.contrib.auth.models import User
from rest_framework import generics
from books.serializers import UserSerializer

class UserList(generics.ListAPIView):
    queryset = User.objects.all()
    serializer_class = UserSerializer


class UserDetail(generics.RetrieveAPIView):
    queryset = User.objects.all()
    serializer_class = UserSerializer

Finally we need to add those views into the API, by referencing them from the URL configuration file. Add the following to the `patterns` list object in `resttutorial/books/urls.py` of your book application.

In [ ]:
url(r'^users/$', views.UserList.as_view()),
url(r'^users/(?P<pk>[0-9]+)/$', views.UserDetail.as_view()),

### Associating books with users

Right now, if we created a book instance, there'd be no way of associating the user that created the book, with the book instance. The user isn't sent as part of the serialized representation, but is instead a property of the incoming request.

The way we deal with that is by rewritting the `post` method of the `BookList` view in `resttutorial/books/views.py`. This will allow us to modify how the instance `save` method is managed and handle any information that is implicit in the incoming request or requested URL.

On the `BookList` view class, replace the post method with the following method:

In [ ]:
def post(self, request, format=None):
    serializer = BookSerializer(data=request.data)
    if serializer.is_valid():
        serializer.save(owner=self.request.user)
        return Response(serializer.data, status=status.HTTP_201_CREATED)
    return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

The `serializer.save()` method will now be passed an additional `owner` field, along with the validated data from the request.

Now that books are associated with the user that created them, let's update our `BookSerializer` to reflect that. Add the following field to the `BookSerializer` definition in `resttutorial/books/serializers.py` (right before the `Meta` class):

In [ ]:
owner = serializers.ReadOnlyField(source='owner.username')

Make sure you also add `owner`, to the `fields` inside the inner `Meta` class of the `BookSerializer` class.

In [ ]:
fields = ('id', 'owner','title', 'author', 'intro', 'price', 'url')

### Adding required permissions to views

Now that book items are associated with users, we want to make sure that only authenticated users are able to create, update and delete book items.

The REST framework includes a number of permission classes that we can use to restrict who can access a given view. In this case the one we're looking for is `IsAuthenticatedOrReadOnly`, which will ensure that authenticated requests get read-write access, and non-authenticated requests get read-only access.

First add the following import in the `resttutorial/books/views.py` module

In [ ]:
from rest_framework import permissions

Then, add the following property to both the `BookList` and `BookDetail` view classes.

In [ ]:
permission_classes = (permissions.IsAuthenticatedOrReadOnly,)

After the addition of the previous functionality, you should notice that you can still carry out GET HTTP requests on your API application either through the Browsable API (http://127.0.0.1:8000/books/) or through the command line:

Remember however that there is no data in the database since we deleted the content at the beginning of today's practical.

Notice also, that through the browsable API (go with your browser to http://127.0.0.1:8000/books/) you don't have the option any longer of uploading data to your web API. 

Also, if you try a POST request through the command line:

You will be notified that you haven't provided the authentication credentials and hence you are not allowed to carry out a POST request.

![](./images/failedAuthentication.png)

To remedy this situation in the browsable API application, we can add a login view for use with the browsable API, by editing the URLconf in our project-level `urls.py` file (`resttutorial/urls.py`). 

Add the following URL pattern to the `urlpatterns` list:

In [ ]:
url(r'^api-auth/', include('rest_framework.urls', namespace='rest_framework')),

The `r'^api-auth/'` part of the pattern can actually be whatever URL you want to use. The only restriction is that the included urls must use the `rest_framework` namespace. In Django 1.9+, REST framework will set the namespace, so you may leave it out.

Now if you open up the browser again and refresh the page (http://127.0.0.1:8000/books/) you'll see a 'Login' link in the top right of the page. If you log in as one of the users you created earlier, you'll be able to programmatically create books instances again.

![](./images/login.png)

Create a few book instances either through the web browser interface or through the command line (you might want to use the same examples we used in the previous practical). 
If we're interacting with the API programmatically (through the command line), we need to explicitly provide the authentication credentials on each request.
Notice that we ddo that by using the http program with the option `-a` standing for **authorization** and our corresponding username and password using the format `userName:password`

Navigate to the `/users/` endpoint of your API, and notice that the representation includes a list of the books primary keys (pks) that are associated with each user, in each user's 'books' field.

![](./images/userList.png)